
# Migrate cross_ws

This notebook will set the modular code in <a href="$../../python/model_migrate" target="_blank">python/model_migrate</a> to path so that it can be imported. Alternately, these modules can be packaged and installed via pip.

This notebook is run in the target experiment environment. In the `cross_ws` scenario, that means that this notebook is run in workspace C. Since in the `cross_ws` scenario, we will need to access a total of three different workspaces, this will have more configurations in the migration instantiation than in other scenarios.

**NOTE**: The migration code currently supported multiple experiments as sources to registered models, however all of those source experiments must come from the same tracking_uri.

In [0]:
%run ./_setup/setup_model_migrate

In [0]:
# Prior to executing this notebook, you must have already created the databricks secrets necessary for working with remote registries.
# That isn't documented here, but can be found at https://docs.databricks.com/en/machine-learning/manage-model-lifecycle/multiple-workspaces.html#set-up-the-api-token-for-a-remote-registry

ws_a_scope = "uc_ml_migrate"
ws_a_prefix = "hls_stat"

ws_b_scope = "uc_ml_migrate"
ws_b_prefix = "genai"

# WS A
print(f"Workspace A Secrets in scope {ws_a_scope}:")
print(f"{ws_a_prefix}-host: " + dbutils.secrets.get(ws_a_scope,f"{ws_a_prefix}-host"))
print(f"{ws_a_prefix}-token: " + dbutils.secrets.get(ws_a_scope,f"{ws_a_prefix}-token"))
print(f"{ws_a_prefix}-workspace-id: " + dbutils.secrets.get(ws_a_scope,f"{ws_a_prefix}-workspace-id"))
print('--------------------------------------')
# WS B
print(f"Workspace B Secrets in scope {ws_b_scope}:")
print(f"{ws_b_prefix}-host: " + dbutils.secrets.get(ws_b_scope,f"{ws_b_prefix}-host"))
print(f"{ws_b_prefix}-token: " + dbutils.secrets.get(ws_b_scope,f"{ws_b_prefix}-token"))
print(f"{ws_b_prefix}-workspace-id: " + dbutils.secrets.get(ws_b_scope,f"{ws_b_prefix}-workspace-id"))

In [0]:
from model_migrate.migration import RegisteredModelMigration

model_name = 'cross_ws'
uc_artifact_location = "dbfs:/Volumes/main/default/experiments"

migration = RegisteredModelMigration(ws_registered_model_name = model_name,
                                     uc_registered_model_name = f'main.default.{model_name}',
                                     ws_tracking_uri = f'databricks://{ws_a_scope}:{ws_a_prefix}',
                                     ws_registry_uri = f'databricks://{ws_b_scope}:{ws_b_prefix}',
                                     uc_tracking_uri = "databricks",
                                     uc_registry_uri = "databricks-uc",
                                     uc_artifact_location = uc_artifact_location)

In [0]:
migration.status()


### Initialize Target Entities

When initializing target entities, what we are doing is creating a place holder experiment run where we will be migrating our latest source experiment runs into. This will be a very fast stage since it is only creating an empty experiment run with a tag `migration_source_run_id` and the cooresponding source experiment run.

In [0]:
migration.initialize_tgt_entities()

In [0]:
#migration.upgrade_exps()

In [0]:
#migration.upgrade_rm()